[Open with Colab](https://colab.research.google.com/github/1never/UEC_AIX_seminar2021/blob/master/UEC_AIX_seminar2021.ipynb)

**メニュー「ランタイム→ランタイムのタイプを変更」**でハードウェアアクセラレータを**GPU**に変更して保存してください．

In [ ]:
# GoogleDriveをマウントする
from google.colab import drive 
drive.mount('/content/drive')
%cd '/content/drive/My Drive'

#学習ファイルのダウンロード
!git clone https://github.com/1never/UEC_AIX_seminar2021.git

#作業用フォルダの作成
!mkdir -p '/content/drive/My Drive/UEC_AIX_seminar2021/'

#青空文庫データ保存用フォルダの作成
!mkdir -p '/content/drive/My Drive/UEC_AIX_seminar2021/aozora_data'

# 学習済みモデル保存用フォルダの作成
!mkdir -p '/content/drive/My Drive/UEC_AIX_seminar2021/bert_data'

#作業用フォルダに移動する
%cd '/content/drive/My Drive/UEC_AIX_seminar2021/'

**マイドライブ＞UEC_AIX_seminar2021>aozora_data** 内に青空文庫からダウンロードしたzipファイルを入れてください．
[青空文庫](https://www.aozora.gr.jp/index.html)

In [ ]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

# transformersのインストール
!pip install transformers==4.4.2 tqdm

In [ ]:
# 作業フォルダに戻る
%cd '/content/drive/My Drive/UEC_AIX_seminar2021/'

In [ ]:
#aozora_dataファルダに移動
%cd '/content/drive/My Drive/UEC_AIX_seminar2021/aozora_data'
#フォルダ内の既存のtxtファイルをすべて削除
!rm *.txt
#フォルダ内のzipファイルを展開する
!unzip '*.zip'
#作業フォルダに移動
%cd '/content/drive/My Drive/UEC_AIX_seminar2021/'

In [ ]:
import re
import glob

#ファインチューニング用のデータを作成する
train_text_list = []
files = glob.glob('./aozora_data/*.txt')
for file in files:
  with open(file, encoding="shift-jis") as f:
    #最終的にtextは句点区切りの文を要素としてもつリストになる
    text = f.read()
    text = re.split('-{55}',text)
    text = re.split('底本：',text[2])
    text = re.sub('《.*》','',text[0])
    text = re.sub('［＃.*］','', text)
    text = re.split("(?<=。)",text)
  #テキストを一文ずつ分割したデータを作成
  for sentence in text[0:-1]:
    if len(sentence):
      train_text_list.append(sentence.strip().replace('\n',''))

#データをtrain.txtとして保存
with open("train.txt", mode='w') as f:
  f.write('\n'.join(train_text_list))
print("Write", len(train_text_list), "lines.")

In [ ]:
%%time
!rm -r ./output
# ファインチューニングの実行
!python ./run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=3 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/ \
    --use_fast_tokenizer=False \
    --block_size 512

**GPT2のみを用いた生成**

1回目の実行は失敗する場合があるので，その場合は2回実行してください．

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

# ファインチューニングしたモデルを用いる
USE_FINETUNED_GPT2 = True
# ファインチューニングしたモデルを用いない
# USE_FINETUNED_GPT2 = False

#候補文をいくつ表示するか
OPTION_NUM = 4

# トークナイザーの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
#モデルの準備
if USE_FINETUNED_GPT2:
  model = AutoModelForCausalLM.from_pretrained("output/")
else:
  model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

u = ""
next_sentence = input("\n>")
log = []
log.append(next_sentence)

while(True):  
  if "exit" == u:
    break
  if "back" == u:
    log.pop()
    next_sentence = re.split("(?<=。)",log[-1])[0]
    print("入力文：")
    print(next_sentence)
  if "log" == u:
    print("ログ：")
    print("\n")
    print("\n".join(log))
    print("\n")
    print("入力文：")
    print(next_sentence)
  if not "。" in next_sentence:
    next_sentence = next_sentence + "。" 
  # 推論
  encoded = tokenizer.encode(next_sentence, return_tensors="pt")
  output = model.generate(encoded, do_sample=True, max_length=100, num_return_sequences=OPTION_NUM)

  sequence_list = []
  for sequence in tokenizer.batch_decode(output):
    sequence = sequence.replace('</s>', '')
    sentence_list = re.split("(?<=。)",sequence)[:-1]
    sequence = "".join(sentence_list)
    sequence_list.append(sequence)
  for i,sequence in enumerate(sequence_list):
    print("[", i,"]",sequence)
  
  u = input("\n>")
  if u.isdecimal():
    choice_sequence = sequence_list[int(u)]
    log.append(choice_sequence[len(next_sentence):])
    next_sentence = re.split("(?<=。)", choice_sequence)[-2]
    print("入力文：")
    print(next_sentence)

  # 1. ">"の右の入力欄に最初の一文を入力します
  # 2. 入力文に続く[0]～[3]までの候補文が表示されます
  # 3. 表示された候補の左の数字を">"の右の入力欄に入力することでその候補の最後の文が次の入力文になります
  # 4. "log"と入力するとこれまでの文章が続けて表示されます
  # 5. "back"と入力すると入力が一つ前まで戻ります
  # 6. "exit"と入力すると終了します



---
**ここからBERTの学習とそれを活用した推論**


In [ ]:
#ライブラリのインストール
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5
!pip install fugashi
!pip install ipadic

In [16]:
import os
import re
import glob
import random

#aozora_dataフォルダ内のtxtファイルをつかってpairデータを作成する
pair_list = []
files = glob.glob('./aozora_data/*.txt')
for file in files:
  with open(file, encoding="shift-jis") as f:
    #最終的にtextは句点区切りの文を要素としてもつリストになる
    text = f.read()
    text = re.split('-{55}',text)
    text = re.split('底本：',text[2])
    text = re.sub('《.*》','',text[0])
    text = re.sub('［＃.*］','', text)
    text = re.split("(?<=。)",text)
    #1～3文と1～3文が対応するペアデータを作成
    for i in range(len(text)-6):
      m = random.randint(1,3)
      n = random.randint(1,3)
      pair_list.append("".join(text[i:i+m])+"\t"+"".join(text[i+m:i+m+n]))

#ペアデータをpair.txtとして保存
with open("pair.txt", mode='w') as f:
  f.write('\n'.join(pair_list))

In [ ]:
write_lines = []
uttrs = []

filename = 'pair.txt'

with open(filename) as f:
    for l in f:
        l = l.strip()
        if "\t" in l:
            # 実際の応答ペアを正解とし，ラベルは1とする．
            write_lines.append('1,' + l.replace("\t", ",") + "\n")
            print('1,' + l.replace("\t", ",") + ',' + "\n")
            # 不正解ペアの作成のため，発話を保存
            uttrs.append(l.split("\t")[0])
            uttrs.append(l.split("\t")[1])
  
# 正解ペアと同じ数だけ不正解ペアを作成
for i in range(len(write_lines)):
    # ランダムな応答ペアを不正解とし，ラベルは0とする．
    write_lines.append('0,' + random.choice(uttrs) + "," + random.choice(uttrs) + "\n")
  
 # 正解ペアと不正解ペアが入ったリストをシャッフルする
random.shuffle(write_lines)
  
index = 0
with open("bert_data/dev.csv", "w") as var_f:
    var_f.write("label,sentence1,sentence2\n")
    # 開発データとしてdev.tsvに200行を書き込む．
    for l in write_lines[:200]:
        var_f.write(l)
        index += 1
index = 0
with open("bert_data/train.csv", "w") as var_f:
    var_f.write("label,sentence1,sentence2\n")
    # 学習データとしてtrain.tsvにのこりを書き込む．
    for l in write_lines[200:]:
        var_f.write(l)
        index += 1
print("Write", len(write_lines[200:]), "lines to train.tsv")

In [ ]:
# max_stepsの値を大きな値に設定することで，より多くのデータで学習できるが，より多くの時間が必要となる．
# 時間がかかるので100に設定しているが，実際は全然足らない．10000以上には設定したい．
!python ./run_glue.py --train_file ./bert_data/train.csv --validation_file ./bert_data/dev.csv  --overwrite_output_dir --overwrite_cache \
--model_name_or_path cl-tohoku/bert-base-japanese-whole-word-masking  --save_steps 100 --max_steps 100 \
--output_dir bert_output/ --do_train --do_eval --per_gpu_train_batch_size 16

In [ ]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class BertEvaluator:
    def __init__(self):
        # 事前学習済みのトークナイザとモデルをロード
        self.tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', do_lower_case=False)
        self.model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=2)
        
        # Google Colabでファインチューニングしたモデルをロード
        self.model.load_state_dict(torch.load("bert_output/pytorch_model.bin", map_location="cpu"))
        self.model.to(device)

    def evaluate(self, user_input, candidate):
        with torch.no_grad():
            # 発話のペアを特徴ベクトルに変換
            tokenized = self.tokenizer([(user_input, candidate)], return_tensors="pt", padding=True)
            input_ids = tokenized["input_ids"].to(device)
            token_type_ids = tokenized["token_type_ids"].to(device)

            # ファインチューニング済みのBERTを用いて特徴ベクトルから2文のスコアを計算
            result = self.model.forward(input_ids, token_type_ids=token_type_ids)
            # softmax関数によりスコアを正規化
            result = F.softmax(result[0], dim=1).cpu().numpy().tolist()

            # 結果を返す．
            return result[0][1]

#BERTを使った評価用クラスの準備
be = BertEvaluator()

# テスト
sentence1 = "おはよう。"
sentence2 = "いい朝ですね。"
sentence3 = "悲しいです。"

score1 = be.evaluate(sentence1, sentence2)
print(f"「{sentence1}」と「{sentence2}」の一貫性:", score1)
score2 = be.evaluate(sentence2, sentence3)
print(f"「{sentence2}」と「{sentence3}」の一貫性:", score2)

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

#文章をいくつ推論するか
RETURN_NUM = 8
#候補文をいくつ表示するか(RETURN_NUMより小さい値にしてください)
OPTION_NUM = 4

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("output/")


u = ""
next_sentence = input("\n>")
log = []
log.append(next_sentence)

while(True):  
  if "exit" == u:
    break
  if "back" == u:
    log.pop()
    next_sentence = re.split("(?<=。)",log[-1])[0]
    print("入力文：")
    print(next_sentence)
  if "log" == u:
    print("ログ：")
    print("\n")
    print("\n".join(log))
    print("\n")
    print("入力文：")
    print(next_sentence)
  if not "。" in next_sentence:
    next_sentence = next_sentence + "。" 
  # 推論
  encoded = tokenizer.encode(next_sentence, return_tensors="pt")
  output = model.generate(encoded, do_sample=True, max_length=100, num_return_sequences=RETURN_NUM)

  sequence_dict = {}
  for sequence in tokenizer.batch_decode(output):
    sequence = sequence.replace('</s>', '')
    sentence_list = re.split("(?<=。)",sequence)[:-1]
    sequence = "".join(sentence_list)
    score = be.evaluate("".join(log[-1]), sequence[len(next_sentence):])
    sequence_dict[sequence] = score
  sequence_dict = sorted(sequence_dict.items(), key=lambda x:x[1], reverse=True)
  for i in range(OPTION_NUM):
    print("[", i,"]", sequence_dict[i][0])
  
  u = input("\n>")
  if u.isdecimal():
    choice_sequence = sequence_dict[int(u)][0]
    log.append(choice_sequence[len(next_sentence):])
    next_sentence = re.split("(?<=。)", choice_sequence)[-2]
    print("入力文：")
    print(next_sentence)

  # 1. ">"の右の入力欄に最初の一文を入力します
  # 2. 入力文に続く[0]～[3]までの候補文が表示されます
  # 3. 表示された候補の左の数字を">"の右の入力欄に入力することでその候補の最後の文が次の入力文になります
  # 4. "log"と入力するとこれまでの文章が続けて表示されます
  # 5. "back"と入力すると入力が一つ前まで戻ります
  # 6. "exit"と入力すると終了します